# Template for analysis work
This template provides the header cells which will create all of the dataframes necessary for analysis using functions stored in project_functions.py

In [1]:
%load_ext autoreload
%autoreload 1
%aimport project_functions
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as plt
import project_functions as pfn

# Used for reading json files from the API
import json
import urllib.request
api_key = '2B92C17D184FEB235C00913E20A82629'

In [2]:
# quick_import will import data from a single, pre-compiled CSV file.
# If not set to True, data will be compiled from source files.

quick_import = True

In [3]:
if quick_import:
    data = pd.read_csv('./master_combined_data.csv')
    data.date = pd.to_datetime(data.date)
    data.set_index('date', inplace=True)
    data.sort_index(inplace=True)
else:
    # Imports annual electricity data for the United States (table10)
    annual = pfn.table10_merge(state_dict=pfn.states, 
                               file_dir="./source_data/Electricity_Data/table10/", 
                               imports_method=3,
                               warnings=False)

    # Import monthly electricity data
    monthly = pfn.load_monthly_electric(state_dict = pfn.states, 
                                        annual_df=annual, 
                                        import_method=3, 
                                        folder_path='./source_data/Electricity_Data/json/', 
                                        warnings=False)

    # Import of land surface weather data
    weather = pfn.load_weather(file_path='./source_data/Weather_Data/MonthlyWeatherDataNOAA_US.txt')

    # Import satellite data
    sat = pd.read_csv('./source_data/GOES_Data/MonthlyAggregate_2005-2009.csv')
    sat.date = pd.to_datetime(sat.date)
    sat.set_index('date', inplace=True)
    sat.sort_index(inplace=True)

    # Merges weather and electricity data
    we = pfn.merge_weather_electric(monthly_df = monthly, weather_df = weather)

    # Merging sat data with existing electric and weather.
    # We now have a single dataframe with all datasets.
    data = pd.merge(we['2005':'2009'], sat, left_index=True, right_index=True)

## This dataframe contains satellite data aggregated by east and west.
## This creates two rows for each month which need to be split and aggregated

# Dataframes for eastern and wester satellite data
east = data[data['Coverage'] == 'East'].copy()
west = data[data['Coverage'] == 'West'].copy()

# Combined, national dataframe
comb = data.groupby(data.index).mean()

# sorting everything
east.sort_index(inplace=True)
west.sort_index(inplace=True)
comb.sort_index(inplace=True)

In [4]:
comb.columns

Index(['gen_tot', 'gen_com', 'gen_ind', 'ret_sales', 'gen_com_ind',
       'direct_use_est', 'net_imports_est', 'losses_est', 'losses_est_perc',
       'losses2', 'losses2_perc', 'losses_perc', 'PCP', 'TAVG', 'PDSI', 'PHDI',
       'ZNDX', 'PMDI', 'CDD', 'HDD', 'SP01', 'SP02', 'SP03', 'SP06', 'SP09',
       'SP12', 'SP24', 'TMIN', 'TMAX', 'ATAVG', 'DTAVG', 'yearmonth', 'hp_sum',
       'ht_sum', 'he_sum', 'hn_sum', 'hp_abs_sum', 'ht_abs_sum', 'he_abs_sum',
       'hn_abs_sum', 'hp_mean', 'ht_mean', 'he_mean', 'hn_mean', 'hp_abs_mean',
       'ht_abs_mean', 'he_abs_mean', 'hn_abs_mean', 'hp_std', 'ht_std',
       'he_std', 'hn_std', 'hp_abs_std', 'ht_abs_std', 'he_abs_std',
       'hn_abs_std', 'hp_min', 'ht_min', 'he_min', 'hn_min', 'hp_abs_min',
       'ht_abs_min', 'he_abs_min', 'hn_abs_min', 'hp_max', 'ht_max', 'he_max',
       'hn_max', 'hp_abs_max', 'ht_abs_max', 'he_abs_max', 'hn_abs_max'],
      dtype='object')

In [5]:
comb.head()

,gen_tot,gen_com,gen_ind,ret_sales,gen_com_ind,direct_use_est,net_imports_est,losses_est,losses_est_perc,losses2,...,he_abs_min,hn_abs_min,hp_max,ht_max,he_max,hn_max,hp_abs_max,ht_abs_max,he_abs_max,hn_abs_max
date,,,,,,,,,,,,,,,,,,,,,
2005-01-01,342162.05171,12469.35811,709.33829,308221.97967,13178.69640,12975.242245,29309.603194,50274.432988,0.135339,20964.829795,...,0.015,0.005,245.5,247.935,103.10,143.00,245.5,247.935,114.00,143.00
2005-02-01,297697.84366,11255.56341,628.49541,278747.81704,11884.05882,11697.088054,43449.872450,50702.811016,0.148624,7252.938566,...,7.790,0.010,151.0,160.685,116.05,62.65,151.0,160.685,116.05,62.65
2005-03-01,316570.00524,12109.71488,671.76976,285959.06987,12781.48464,12621.701599,7011.165361,25000.399132,0.077262,17989.233771,...,6.150,0.000,151.0,153.180,118.50,58.80,151.0,153.180,118.50,61.60
2005-04-01,288675.59475,11489.75673,636.62078,263327.77507,12126.37751,11961.203313,-10232.090923,3154.525445,0.011329,13386.616367,...,11.560,0.020,174.0,176.970,125.65,37.15,174.0,176.970,125.65,37.15
2005-05-01,314101.50257,11832.74246,666.08129,272599.51797,12498.82375,12195.768695,-27132.245147,2173.970758,0.007576,29306.215905,...,0.520,0.005,190.0,281.335,265.50,78.10,190.0,281.335,265.50,94.70
